In [21]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [27]:
month, day, year = 11, 5, 2019
url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month='+str(month)+'&day='+str(day)+'&year='+str(year)
#url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month=11&day=5&year=2019'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')


In [7]:
soup.find('td').find('a').text

'Michigan State'

In [53]:
scores = soup.find_all('td')
#for score in scores:
#    print(score)


In [51]:
i=0
for score in scores:
    if i%6 in [0,3]:
        print(score.find('a').text)
    if i%6 in [1,4]:
        print(int(score.text))
    i+=1
    if i == 6:
        break

Michigan State
62
Kentucky
69


In [36]:
visitor_neutral = []
home_neutral = []
v_score = []
h_score =[]
dates = []
date =str(month)+'/'+str(day)+'/'+str(year)
i=0
for info in scores:
    if i%6 == 0:
        visitor_neutral.append(info.find('a').text)
    elif i%6 == 3:
        home_neutral.append(info.find('a').text)
    elif i%6 == 1:
        v_score.append(int(info.text))
    elif i%6 == 4:
        h_score.append(int(info.text))
    else:
        dates.append(date)
    i+=1

In [37]:
print(visitor_neutral[0], v_score[0], home_neutral[0], h_score[0])

Michigan State 62 Kentucky 69


In [48]:
df = pd.DataFrame(zip(dates, visitor_neutral, v_score, home_neutral, h_score), 
               columns =['Date','visitor_neutral', 'PTS','home_neutral','PTS']) 
df['Date']= pd.to_datetime(df['Date'])
df.head()

,Date,visitor_neutral,PTS,home_neutral,PTS
0,2019-11-05,Michigan State,62,Kentucky,69
1,2019-11-05,Kansas,66,Duke,68
2,2019-11-05,Louisville,87,Miami (FL),74
3,2019-11-05,North Florida,59,Florida,74
4,2019-11-05,Holy Cross,71,Maryland,95


In [58]:
month, day, year = 11, 5, 2019
visitor_neutral = []
home_neutral = []
v_score = []
h_score =[]
dates = []
master_df = pd.DataFrame()
date =str(month)+'/'+str(day)+'/'+str(year)
while(day<=30):
#reset lists
    visitor_neutral = []
    home_neutral = []
    v_score = []
    h_score =[]
    dates = []
    date =str(month)+'/'+str(day)+'/'+str(year)
#Get score page for date
    url = 'https://www.sports-reference.com/cbb/boxscores/index.cgi?month='+str(month)+'&day='+str(day)+'&year='+str(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    scores = soup.find_all('td')
#get lists of all games / scores
    i=0
    for info in scores:
        if i%6 == 0:
            visitor_neutral.append(info.find('a').text)
        elif i%6 == 3:
            home_neutral.append(info.find('a').text)
        elif i%6 == 1:
            v_score.append(int(info.text))
        elif i%6 == 4:
            h_score.append(int(info.text))
        else:
            dates.append(date)
        i+=1
#make dataframe of lists
    df = pd.DataFrame(zip(dates, visitor_neutral, v_score, home_neutral, h_score), 
               columns =['Date','visitor_neutral', 'PTS','home_neutral','PTS']) 
    df['Date']= pd.to_datetime(df['Date'])
#combine with master dataframe
    master_df = pd.concat([master_df, df])
# increment day
    day+=1

    

    

In [60]:
master_df.head(1)

,Date,visitor_neutral,PTS,home_neutral,PTS
0,2019-11-05,Michigan State,62,Kentucky,69


In [61]:
master_df.tail(1)

,Date,visitor_neutral,PTS,home_neutral,PTS
63,2019-11-30,Youngstown State,72,Central Michigan,88
